In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive

/content/drive


In [ ]:
cd MyDrive

/content/drive/MyDrive


In [ ]:
cd cdQA-bert_qa_vGPU

/content/drive/MyDrive/cdQA-bert_qa_vGPU


In [ ]:
pip install -e .

Obtaining file:///content/drive/MyDrive/cdQA-bert_qa_vGPU
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 12.1 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=1fcb224c0b549991c0a9d357460242bb61467de434704a21d73b351517000be1
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling P

In [ ]:
pip install cdqa

# train 데이터 다운

In [ ]:
import os
import torch
import joblib
from cdqa.reader.bertqa_sklearn import BertProcessor, BertQA
from cdqa.utils.download import download_squad_assets # 한국어 데이터라 안해도 ㄱㅊ

/content/drive/MyDrive/cdQA-bert_qa_vGPU/cdqa/reader/bertqa_sklearn.py:36: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# download_squad_assets() 영어만 해당

# reader Bert training

In [ ]:
# pre-process examples
train_processor = BertProcessor(do_lower_case=True, is_training=True)
train_examples, train_features = train_processor.fit_transform(X='data/train-v1.1.json') # korean.csv는 korquad 데이터로 학습시켜야함

# train the model
# train_batch_size를 4보다 크게 할 경우, colab에서 제공하는 GPU로는 메모리가 부족해서 에러가 발생
reader = BertQA(train_batch_size=4,
               learning_rate=3e-5,
               num_train_epochs=2,
               do_lower_case=True,
               fp16=False,
               output_dir='models')

reader.fit(X=(train_examples, train_features))

100%|██████████| 407873900/407873900 [00:12<00:00, 32747066.48B/s]


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/22161 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_pretrained_bert/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1519.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


Iteration:   0%|          | 0/22161 [00:00<?, ?it/s]

BertQA(do_lower_case=True, learning_rate=3e-05, num_train_epochs=2,
       output_dir='models', train_batch_size=4)

In [ ]:
# send it to GPU
reader.model.to('cuda')
reader.device = torch.device('cuda')

# save it locally
joblib.dump(reader, os.path.join(reader.output_dir, 'bert_qa_squad_vGPU.joblib'))

['models/bert_qa_squad_vGPU.joblib']

## <발견사항>
- 1) V100 GPU 사용 권장
- 2) Colab Pro에서도 batch_size=4 이상부터는 colab에서 오류가 남

## <셀 출력과 함께 저장하는 방법>
수정 > 노트 설정 > 이 노트를 저장할 때 코드 셀 출력 생략(이걸 선택 취소해야함)